### Game mit Eventloop
Wir benutzen einen Eventloop, um ein einfaches Spiel zu programmieren,
bei dem der Spieler einen Linienzug zeichnet. Die Linie setzt sich jeweils ein paar Pixel in
Richtung der zuletzt gedrücken Pfeiltaste fort.
Kreuzt sich die Linie oder erreicht sie den Rand der Leinwand, endet das Spiel.

Wir denken uns die Linie als Schlange, die sich in einem NxN Gitter bewegt und immer länger wird.
Der Spielzustand ist in einem Dict gespeichert.
Die Position des Kopfes `(col, row)` ist dem Schlüssel `head` zugewiesen,
die von restlichen Körper besetzten Felder sind als
NxN Tabelle repräsentiert, die dem Schlüssel `body` zugewiesen ist
Besetzte Felder sind mit `True` markiert, leere mit `False`.
Die Spielkomponente hat eine Funktion `move(dx, dy)`, welche die aktuelle Kopfposition `(x, y)`,
auf `(x+dx, y+dy)` updated und `(x, y)` als besetzt markiert.

Der Spielzustand wird auf der Leinwand dargestellt. Dabei wird in der mitte der Feldes eine
Linie in Bewegungsrichtung gezeichnet.

Wird eine Pfeiltaste gedrückt, wird diese als erstes Element in eine Liste
`event_queue` eingefügt. 
Ist der Eventqueue  nicht leer, entfernt 
der Eventloop die zuletzt gedrückt Taste, andernfalls wird
die zuletzt gedrückt Taste beibehalten. Die Taste wird in der Variable `key` gespeichert.
Dann wird die Funktione `game.move` mit den Argumenten `key_move[key]` aufgerufen.

Der Event-Loop wird mit Hilfe der Funktion `threading.Timer`.
Nachstehend definierte Funktion `event_loop` 
macht sich zu einem Thread, der mit etwas Verzögerung startet.
Und dieser Thread macht aus der Funktion erneut einen solchen Thread.
Dieser Vorgang wird gestoppt, wenn das `stop_event` gesetzt wird.


```python
stop_event = threading.Event()
event_queue = []

key_move = {'ArrowUp': (0, -1),
            'ArrowDown': (0, 1),
            'ArrowLeft': (-1, 0),
            'ArrowRight': (1, 0),
            }

def event_loop(last_event=None, count=0):
    if event_queue:
        key = event_queue.pop()
    else:
        key = last_event

    if key in key_move:
        dx, dy = key_move[key]
        game.move(dx, dy)

    if not stop_event.is_set():
        thread = threading.Timer(1, event_loop, args=(key, count+1))
        thread.name = f'Eventloop-{count}'
        thread.start()            
```


***
Die Spielkomponente unseres Spiels ist in `game.py` implementiert
***

In [ ]:
import game

# Right, Up, Left, Down -> Game over
moves = [(1, 0), (0, 1), (-1, 0)] + [(0, -1)]

game.new_game(nrow=5, ncol=5)
for dx, dy in moves:
    game.move(dx, dy)

game.state

***
Eventloop testen.
***

In [ ]:
#%%file eventloop.py
import threading
import game


stop_event = threading.Event()
event_queue = []


key_move = {'ArrowUp': (0, -1),
            'ArrowDown': (0, 1),
            'ArrowLeft': (-1, 0),
            'ArrowRight': (1, 0),
            }


def event_loop(last_event=None, count=0):
    if event_queue:
        key = event_queue.pop()
    else:
        key = last_event

    if key in key_move:
        dx, dy = key_move[key]
        game.move(dx, dy)

    if not stop_event.is_set():
        thread = threading.Timer(1, event_loop, args=(key, count+1))
        thread.name = f'Eventloop-{count}'
        thread.start()

In [ ]:
game.new_game()

In [ ]:
event_queue = ['ArrowUp'] + ['ArrowRight'] * 3
stop_event.clear()
event_loop()

In [ ]:
# laeuft der Eventloop Thread noch?
[thread.name for thread in threading.enumerate()[8:]]

In [ ]:
# Eventloop stoppen
stop_event.set()

Game ohne Eventloop und ohne Selbstupdate

In [ ]:
import game
from ipywidgets import Output
from ipycanvas import Canvas
from IPython.display import display


layout = {'border': '1px solid black'}
out = Output(layout=layout)
canvas = Canvas(width=100, height=100, layout=layout)

state = {'pos': None}
key_move = {'ArrowUp': (0, -1),
            'ArrowDown': (0, 1),
            'ArrowLeft': (-1, 0),
            'ArrowRight': (1, 0),
            }


def grid2canvas(x, y, ncol, nrow):
    return x*canvas.width/ncol, y*canvas.height/nrow


@out.capture(clear_output=True)
def on_key_down(key, *flags):
    print(key)
    if key.startswith('Arrow'):
        dx, dy = key_move[key]
        game.move(dx, dy)
        move(game.state['head'], game.state['size'])
        if game.is_game_over():
            game_over()
    if key == 'n':
        game.new_game()
        new_game(game.state['head'], game.state['size'])


def new_game(head, size):
    x, y = grid2canvas(*head, *size)
    canvas.clear()
    canvas.fill_circle(x, y, 2)
    state['pos'] = x, y


def game_over():
    canvas.fill_text('Game over!', 10, 50)


def move(head, size):
    x, y = grid2canvas(*head, *size)
    x0, y0 = state['pos']
    canvas.stroke_lines([(x0, y0), (x, y)])
    state['pos'] = (x, y)


canvas.on_key_down(on_key_down)
display(canvas, out)
canvas.focus()

### Aufgabe